# Construcción de la Base de Datos. 

Ahora comenzaremos con la construcción de una base de datos que contenga las tablas mencionadas con anterioridad. Así que primero vamos a descargarlas de la carpeta format_data en results.

In [330]:
import os
from download_data import tables_tidy_sort as tt
import numpy as np
import pandas as pd

folder_path = "..\\results\\format_data"
tables = os.listdir(folder_path)


Ya los tenemos enlistdos. Ahora debemos recurrir a `tables_tidy.py` para recuperar los indices multiples. 

In [331]:
"""
def collapse_multiIndex(multi: pd.MultiIndex | list):
    if type(multi) == list:
        return multi
    n = multi.nlevels
    mi = []
    for j in range(n):
        level = []
        for i in range(len(multi)):
            level.append(multi[i][j])
        mi.append(level)
    return mi
"""

mi_sonora = tt.mi

Eso lo realizaremos mediante la función collapse_... . Ahora bien. Debemos ver cual es el indice más grande que tenemos

In [332]:
sonora_levels = [mi.nlevels if type(mi) == pd.MultiIndex else 1 for mi in mi_sonora]
max_level = np.array(sonora_levels).max()
max_level

5

Entonces el nivel más grande es el 5. Entonces los que tengan  menos debemos de agregarles las que faltan.

In [333]:
def add_levels(multi: pd.MultiIndex | list, fill: str, new_max: int):
    if type(multi) == list:
        nl = 1
    else:
        nl = multi.nlevels
    add_levels = new_max - nl
    new_l = []
    level_length = len(multi)
    for i in range(add_levels):
        if i == 0:
            level = level_length * [fill]
        else:
            level = level_length * [""]
        new_l.append(level)
    return new_l


def tuples_to_list(multi: pd.MultiIndex):
    m = multi.nlevels
    n = len(multi)
    mi_list = []
    for i in range(m):
        aux = []
        for j in range(n):
            aux.append(multi[j][i])
        mi_list.append(aux)
    return mi_list


def normalize_df(multi, text, nl):
    mi = tuples_to_list(multi)
    new_mi = add_levels(multi, text, nl)
    new_mi += mi
    new_mi = pd.MultiIndex.from_arrays(new_mi)
    return new_mi

Como etiqueta agregaremos los años de extraccion de información, (que de forma conveniente estan en el nombre del archivo). Entonces vamos a extraerlos. 

In [334]:
import regex as re

names = [os.path.splitext(os.path.basename(file))[0] for file in tables ]
years = [re.findall('\d{4}',file) for file in names]
years = [a[0] if len(a) > 0 else 'Sin año' for a in years]

In [335]:
dataframes = []
for i in range(len(tables)):
    u = pd.read_excel(folder_path + "\\" + tables[i], skiprows=sonora_levels[i], header=None)
    dataframes.append(u)


dfs = []
for df in dataframes:
    df = df.set_index(df.columns[0], drop=True)
    ni = df.index.set_names('Municipios')
    df.index = ni
    dfs.append(df)

Municipios = dfs[0].index

In [336]:
new_dfs = []
for i in range(len(tables)):
    di = dfs[i]
    mi = mi_sonora[i]
    if type(mi) == pd.MultiIndex:
        mi = normalize_df(mi, years[i], max_level + 1)
    df = pd.DataFrame(index = Municipios, columns = mi)
    # df.columns = list(range(len(dfs[i].columns)))
    new_dfs.append(df)


for k,data in enumerate(dfs):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            new_dfs[k].iloc[i, j] = data.iloc[i, j]

In [337]:
abs_dfs = []
per_dfs = []
other_dfs = []
for k, df in enumerate(new_dfs):
    test_cols = df.columns
    col_abs = []
    col_per = []
    col_other = []
    for col in test_cols:
        if col[-1] == 'Absoluto':
            col_abs.append(col)
        else:
            if col[-1] == 'Porcentaje':
                col_per.append(col)
            else:
                col_other.append(col)

    df_abs = df[col_abs]
    df_per = df[col_per]
    df_other = df[col_other]
    try:
        df_abs = df_abs.droplevel(5, axis = 1)
        df_per = df_per.droplevel(5, axis = 1)
    except:
        continue
    abs_dfs.append(df_abs)
    per_dfs.append(df_per)
    other_dfs.append(df_other)

Finalmente, lo agregamos a las columnas.

In [338]:
for i in range(len(abs_dfs)):
    if i == 0:
        final_abs_df = abs_dfs[i]
    else:
        final_abs_df = pd.concat([final_abs_df, abs_dfs[i]], axis = 1)

for i in range(len(per_dfs)):
    if i == 0:
        final_per_df = per_dfs[i]
    else:
        final_per_df = pd.concat([final_per_df, per_dfs[i]], axis = 1)

absf_df = final_abs_df.iloc[1:]
perf_df = final_per_df.iloc[1:]

In [339]:
absf_df.head()

2022                                           \
                                                               
                                                               
                                                               
            Total 1 Accidentes fatales Accidentes no fatales   
Municipios                                                     
Aconchi         0.0                0.0                   0.0   
Agua Prieta   133.0                4.0                  59.0   
Alamos         67.0                1.0                  21.0   
Altar         116.0                0.0                   0.0   
Arivechi        4.0                0.0                   0.0   

                                                                       \
                                                                        
                                                                        
                                                                        
            Accidentes con sólo daños materiales    Total Automóviles   
Municipios                                                              
Aconchi                                      0.0    936.0       437.0   
Agua Prieta                                 70.0  48087.0     31678.0   
Alamos                                      45.0   4627.0      1918.0   
Altar                                      116.0   2799.0      1541.0   
Arivechi                                     4.0    321.0        99.0   

                                                                      \
                                                                       
                                                                       
                                                                       
            Camiones para pasajeros Camiones y camionetas para carga   
Municipios                                                             
Aconchi                         4.0                            480.0   
Agua Prieta                   226.0                          15774.0   
Alamos                         46.0                           2642.0   
Altar                           9.0                           1240.0   
Arivechi                        1.0                            219.0   

                             Sin año  ...                   2022  \
                                      ...                          
                                      ...                          
                         Localidades  ... Servicios de Salud (1)   
            Motocicletas       Total  ...      Poblacion Usuaria   
Municipios                            ...                          
Aconchi             15.0        25.0  ...                 2518.0   
Agua Prieta        409.0       261.0  ...                88416.0   
Alamos              21.0       529.0  ...                24524.0   
Altar                9.0       351.0  ...                 9213.0   
Arivechi             2.0        97.0  ...                 1125.0   

                                                                             \
                                                                              
                                                                              
            Tasa de mortalidad Tasa de mortalidad infantil Unidades médicas   
                                                                              
Municipios                                                                    
Aconchi                      -                           -              2.0   
Agua Prieta                  -                           -              8.0   
Alamos                       -                           -             21.0   
Altar                        -                           -              3.0   
Arivechi                     -                           -              2.0   

                                                                      \
                      

In [340]:
other_df = df_other.iloc[1:]
other_df

Sin año                                     \
                                                                             
                                                                             
                            Coordenadas                                      
                                Latitud               Longitud               
                                Min (°)    Max (°)     Min (°)     Max (°)   
Municipios                                                                   
Aconchi                       29.683333  29.883333       110.0  110.433333   
Agua Prieta                        30.7      31.35  108.683333      109.75   
Alamos                        26.383333  27.783333  108.416667      109.25   
Altar                         30.516667  31.666667       111.6  112.283333   
Arivechi                      28.733333  28.983333  108.833333  109.266667   
..                                  ...        ...         ...         ...   
Villa Pesqueira               28.783333       29.5  109.783333  110.133333   
Yécora                        28.166667      28.65  108.466667  110.133333   
Gral. Plutarco Elías Calles   31.316667  32.083333       112.3  113.516667   
Benito Juárez                 26.966667  27.233333  109.716667  109.983333   
San Ignacio Río Muerto        27.116667  27.533333  110.133333       110.6   

                                             \
                                              
                                              
                                              
                            Altitud           
                            Min (m) Max (m)   
Municipios                                    
Aconchi                       500.0  2200.0   
Agua Prieta                   800.0  2500.0   
Alamos                         50.0  2000.0   
Altar                         200.0  1700.0   
Arivechi                      400.0  2200.0   
..                              ...     ...   
Villa Pesqueira                50.0  1700.0   
Yécora                        300.0  2300.0   
Gral. Plutarco Elías Calles   100.0  1400.0   
Benito Juárez                   0.0   100.0   
San Ignacio Río Muerto          0.0   100.0   

                                                                                
                                                                                
                                                                                
                                                                  Colindancias  
                                                                                
                                                                                
Municipios                                                                      
Aconchi                      Colinda al norte con los municipios de San Fel...  
Agua Prieta                  Colinda al norte con los Estados Unidos de Amé...  
Alamos                       Colinda al norte con el municipio de Quiriego ...  
Altar                        Colinda al norte con el municipio de Caborca y...  
Arivechi                     Colinda al norte, este, sur y oeste con el mun...  
..                                                                         ...  
Villa Pesqueira              Colinda al norte con los municipios de Baviáco...  
Yécora                       Colinda al norte con los municipios de Bacanor...  
Gral. Plutarco Elías Calles  Colinda al norte con el municipio de Puerto Pe...  
Benito Juárez                Colinda al norte con los municipios de Cajeme ...  
San Ignacio Río Muerto       Colinda al norte con los municipios de Guaymas...  

[72 rows x 7 columns]

## CVEGEO

La clave geográfica (CVEGEO) es un número único asignado a cada region definida dentro del país. En nuestro caso trabajaremos con Sonora. 

In [341]:
cvegeo = pd.read_excel('..\\results\\cvegeo\\cvegeo.xlsx')

absf_df.loc[:, 'CVEGEO'] = cvegeo['CVE'].unique().tolist()
perf_df.loc[:, 'CVEGEO'] = cvegeo['CVE'].unique().tolist()




C:\Users\Tamara\AppData\Local\Temp\ipykernel_9368\4103526707.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  absf_df.loc[:, 'CVEGEO'] = cvegeo['CVE'].unique().tolist()
C:\Users\Tamara\AppData\Local\Temp\ipykernel_9368\4103526707.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_df.loc[:, 'CVEGEO'] = cvegeo['CVE'].unique().tolist()


Ahora, debemos colocarlo al comienzo.

In [342]:
# Obtener el nombre de la última letra en el MultiIndex
last_letter = absf_df.columns.levels[0][-1]
# Mover la última letra al comienzo del MultiIndex
absf_df = absf_df[[last_letter] + [col for col in absf_df.columns.levels[0] if col != last_letter]]
perf_df = perf_df[[last_letter] + [col for col in perf_df.columns.levels[0] if col != last_letter]]

absf_df.to_excel('..\\results\\database\\absolute_database.xlsx')
perf_df.to_excel('..\\results\\database\\percentage_database.xlsx')

acols = list(absf_df.columns)
pcols = list(perf_df.columns)


acols.remove(('2020', '', '', '', 'Tenencia de la vivienda'))
pcols.remove(('2020', '', '', '', 'Tenencia de la vivienda'))

absf_df = absf_df.drop(columns=('2020', '', '', '', 'Tenencia de la vivienda'))
perf_df = perf_df.drop(columns=('2020', '', '', '', 'Tenencia de la vivienda'))


In [343]:
descriptor = pd.read_excel("..\\data\\descriptor.xlsx")
descriptor.set_index('Némonico',drop=True,inplace=True)
new_cols = list(descriptor.index)
pe = ['CVEGEO'] + new_cols[:-1]
ab = ['CVEGEO'] + new_cols

ch_ab_columns = {absf_df.columns[i] : ab[i] for i in range(len(ab))}
ch_pe_columns = {perf_df.columns[i] : pe[i] for i in range(len(pe))}


absf_df.columns = ab
perf_df.columns = pe

Una vez agregado el CVEGEO, que se usará más adelante. Vamos con las correcciones.

# Correciones a la Base de Datos. 

La base de datos ya fue terminada. Sin embargo, esta presento fallos que debemos corregir. 

* Atender los comentarios en color violeta que están en la pestaña “Diccionario de variables”.
* Remover caracteres no numéricos de campos con variables numéricas (ej. “ND”, “-“) desde la base original.
* Generar una salida por municipio con estadísticas de resumen para las variables (N, max, min, prom, med, sd)
* Generar matrices de correlación entre las variables del mismo año, apoyándote en la pauta del ejemplo de gráfica adjunta. Observé que el conjunto de variables relacionadas con “entorno urbano” y de “establecimientos” tienen baja correlación en ese nivel con variables sociodemográficas, estas las puedes omitir, yo te indico más o menos cuáles serían.
* Para generar visualizaciones, utiliza los nemónicos cortos, me funcionó bien. Pues usar quizás una nomenclatura que sea más indicativa, yo usé por practicidad esta de “var…” pero es difícil identificar rápido cuál es la relación con cual.

Por comodidad reharemos las tablas. Extrayendo los años y el tipo de dato.

Mostraremos el proceso con la tabla de valores absolutos. Pero se aplicaria de igual forma con los porcentuales.

In [344]:
def check_float(df: pd.DataFrame):
    n = len(df.columns)
    mod_cols = []
    for j in range(n):
        try:
            df.iloc[:, j] = df.iloc[:,j].astype(float)
        except:
            mod_cols.append(df.columns[j])
    return mod_cols

mod_abs = check_float(absf_df)
mod_abs

C:\Users\Tamara\AppData\Local\Temp\ipykernel_9368\1584852348.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, j] = df.iloc[:,j].astype(float)


['TCP9000',
 'TCP0010',
 'TCP1020',
 'DFT',
 'TPE',
 'TD',
 'TMOR',
 'TMORI',
 'SEGSEP']

Con las funciones anteriores, podemos ver cuales columnas no se pueden transformar en flotantes. Ahora debemos corregirlas.

In [345]:
def correct_not_float(modify_cols: list, df: pd.DataFrame):
    for col in modify_cols:
        if (df.loc[:, col] == '-').sum() == 72:
            df = df.drop(columns = col)
    return df

df = correct_not_float(mod_abs, absf_df)
mod_df = check_float(df)
mod_df

['DFT', 'SEGSEP']

Con la función anterior, borramos las que son totalmente '-', que no nos otorgan información. Quedando únicamente dos.

In [346]:
df.loc[:, mod_df[0]] = [s.replace(" ","") for s in absf_df.loc[:, mod_df[0]]]
mod2_df = check_float(df)
mod2_df

C:\Users\Tamara\AppData\Local\Temp\ipykernel_9368\1584852348.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, j] = df.iloc[:,j].astype(float)


['SEGSEP']

Finalmente, solo nos queda una columna. Sin embargo, esta no será transformada. Ya que posee datos que no son '-'.

In [347]:
df['SEGSEP'].value_counts()

ND        69
234752     1
278406     1
70703      1
Name: SEGSEP, dtype: int64

Entonces, solo remplazarmemos por espacios en blanco. 

In [348]:
df = df.replace('ND',pd.NA)
df['SEGSEP'].value_counts()

234752    1
278406    1
70703     1
Name: SEGSEP, dtype: int64

Finalmente, guardamos el archivo. 

In [349]:
df['CVEGEO'] = df['CVEGEO'].astype(str)
df.to_excel('..\\results\\database\\abs_float.xlsx')
absolute_df = df.copy()

Ahora checaremos los datos porcentuales. 

In [358]:
mod_per = check_float(perf_df)

for col in mod_per:
    u = (perf_df.loc[:, col] == '-').sum()
    if u == 72:
        perf_df = perf_df.drop(columns=col)
mod_per = check_float(perf_df)
df = perf_df.copy()

In [359]:
df = df.replace('ND',pd.NA)
df['SEGSEP'].value_counts()

92.3    1
46.5    1
42.9    1
Name: SEGSEP, dtype: int64

In [360]:
df['CVEGEO'] = df['CVEGEO'].astype(str)

df.dtypes


CVEGEO      object
PT1990     float64
PT2000     float64
PT2010     float64
PT2020     float64
            ...   
AGEBURB    float64
AGEBRUR    float64
MANURB     float64
MANRUR     float64
SUP        float64
Length: 141, dtype: object

In [361]:
df.to_excel('..\\results\\database\\per_float.xlsx')
percentage_df = df.copy()

# Incorporando Mapas.

In [362]:
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import requests
import zipfile
import fiona 


# Paleta de Colores
colors = {
    "green": '#097275',
    "silver": '#A8A8A7',
    "brown": '#7C5C15',
    "orange": '#C96C1C',
    "gold": '#CCB656',
    "cherry": '#832347'
}

isaf = [c[1] for c in colors.items()]

isaf = sns.color_palette(isaf)

# Descarga de Poligonos.
shapes_url = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/26_sonora.zip"
shapes_archivo = "data\\26_sonora.zip"

r = requests.get(shapes_url, allow_redirects=True)
open("..\\data\\Sonora.zip", 'wb').write(r.content)

with zipfile.ZipFile("..\\data\\Sonora.zip", 'r') as zip_ref:
    zip_ref.extractall("..\\data\\geoSonora")



shape = fiona.open("..\\data\\geoSonora\\conjunto_de_datos\\26mun.shp")
Sonora = gpd.GeoDataFrame.from_features([feature for feature in shape])
# Get the order of the fields in the Fiona Collection; add geometry to the end
columns = list(shape.meta["schema"]["properties"]) + ["geometry"]
# Re-order columns in the correct order
Sonora = Sonora[columns]

En Sonora tenemos la base de datos de los polígonos para la creación de mapas. Ahora, debemos incorporar nuestros dataframes.

In [384]:
gd_son_abs = gpd.GeoDataFrame(geometry=Sonora['geometry'],crs='epsg:3857')
gd_son_pers = gpd.GeoDataFrame(geometry=Sonora['geometry'],crs='epsg:3857')
